<a href="https://colab.research.google.com/github/bigflyy/DeepLearningWithPython/blob/main/2LinearClassifierTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import seaborn as sns

## Tensors

In [2]:
a = tf.ones((2, 2))
b = tf.square(a)
c = tf.sqrt(a)
d = b + c
e = tf.matmul(a, b)
e *= d

In [4]:
e *= d

In [6]:
e[0,0] = 4

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

This proves it is immutable tho operations on tensors do work so they can change

## Gradient Tape

In [9]:
input_var = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
  result = tf.square(input_var)
gradient = tape.gradient(result, input_var)
gradient

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

Interesting that it has absolutely no variables, just numbers, and it computes it nonetheless...